# K Means Example

In this exercise, you will code up your implementation of K Means clustering. Included is a rough algorithm. Make sure you understand it and are able to express it in your head before moving on.

```
Choose `n_k`: the number of clusters
Choose `n_iter`: the number of total iterations to run k means
Set cluster_labels: this is an array or series that is the length of your input data rows. 
  This array corresponds to the predictions or label associated with each data row.

Randomly select n_k of your data points to be the initial centroid positions.

Now, for each iter in n_iter:

  For each point in your data points:
     Get the squared distance of the point to all of the centroids. 
     (i.e. squared l2 norm, or squared euclidean distance)
     Assign the cluster_label of this point to the label of the closest centroid
 
  For each centroid in n_k:
     Grab all the points associated with this cluster.
     Set the centroid's new position as the mean of all of these points along each dimension
  
  Return cluster_labels

```

In addition, play with this link until you feel comfortable with the algorithm in your head

http://www.naftaliharris.com/blog/visualizing-k-means-clustering/


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# Dont need to touch

# Generate Fake Data
centroids = [ [1.0,1.5],[2.5,2.8],[1.0,4.0] ]

data = pd.DataFrame(np.zeros((150,2)), columns=['x1','x2'])

for cidx,c in enumerate(centroids):
    for idx in range(50):
        data.ix[50*cidx + idx,0:2] = [c[0],c[1]] + np.random.normal(0,.4,2)

X = data[ ['x1','x2'] ]

In [ ]:
# Dont need to touch

def draw2DClusters(data, labels=None, title="Clustered Points"):
    """
    draw2DClusters takes in a dataframe or matrix with m rows, and 2 columns (i.e. two features)
    as well as a series of integer labels that incidate which cluster they belong to
    
    This function will plot a scatter with the appropriate scatters labeled. If there are no labels specified
    then will plot all with the same color
    """
    if labels is None:
        labels = np.zeros(data.shape[0])
    
    assert(data.shape[1] == 2) # Input dataframe or matrix must have 2 columns to properly plot this
    labels = labels.astype(int)
    color_map = [ plt.cm.rainbow(i) for i in np.linspace(0, 1.0, len(np.unique(labels)))]

    for cidx, c in enumerate(np.unique(labels)):
        X_sub = X.ix[ labels == c, :]
        plt.scatter(X_sub.iloc[:,0],X_sub.iloc[:,1],s=50,alpha=0.5,c=color_map[cidx])
        plt.xlabel(r"Feature $x_1$"), plt.ylabel(r"Feature $x_2$"), plt.title(title)

        centroid = np.mean(X_sub)
        plt.plot(centroid[0],centroid[1],'*',markersize=30,alpha=0.75,c=color_map[cidx])

In [ ]:
# Do touch

from scipy.spatial import distance

def kmeans(X, n_k, n_iter, standardize=True):
    '''
    X: Dataframe or matrix each column being a feature
    n_k: Number of clusters
    n_iter: Number of iterations to go through
    standardize: Should each feature be standardized prior to clustering
    
    Returns a series of the class idx labels
    '''
    X = X.copy()
    
    # if standardize set true, then standardize by each column
    if standardize:
        col_std = np.sqrt(np.var(X,axis=0,keepdims=True))
        X = X / col_std

    # Set up return array
    cluster_labels = np.zeros(len(X))

    # TODO: Sample n_k points from the X dataframe. Choose these as the clusters
    # It may help to save centroids as a dict with key = label, value = centroid point

    
    
    ## End TODO
    
    for itr in range(n_iter):
        for idx in range(len(X)):
            
            #TODO
            # Compute the distance of each point to each centroid. 
            # Set cluster_labels of idx as the closest centroid
            # It may help to use the euclidean function in distance
            # Remember to use the SQUARED euclidean distance! 
            

                    
            #End TODO

        #TODO
        # For each cluster, compute the new centroid as the mean of the points
        # that are a part of the cluster.

        
        
        # End TODO
        

    return cluster_labels


In [ ]:
# Run this after you are sure of your implementation

# Test 1 -- Output should look similar to the sklearn impl, par for the actual color choices
plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
cluster_labels = kmeans(X,3,10, standardize=True)
draw2DClusters(X,cluster_labels,title="Your K Means")


plt.subplot(1,2,2)
from sklearn import cluster
kn = cluster.KMeans(n_clusters=3)
kn.fit(X)
cluster_labels = kn.predict(X)
draw2DClusters(X,cluster_labels,title="SKLearn KMeans")


In [ ]:
# Run this after you are sure of your implementation


# Test 2 - This tests how your impl looks with 1 vs. 3 vs. 10 clusters
#1 Cluster should have centroid at the center
# 3 Cluster should have a centroid over each natural cluster
# 10 cluster should have 3 or 4 clusters roughly located around each natural cluster

plt.figure(figsize=(15,5))
plt.subplot(1,3,1)
cluster_labels = kmeans(X,1,10, standardize=True)
draw2DClusters(X,cluster_labels,title="1 Cluster")

# Generate 3 clusters and draw
plt.subplot(1,3,2)
cluster_labels = kmeans(X,3,10, standardize=True)
draw2DClusters(X,cluster_labels,title="3 Clusters")


# Generate 10 Clusters and draw
plt.subplot(1,3,3)
cluster_labels = kmeans(X,10,10, standardize=True)
draw2DClusters(X,cluster_labels,title="10 Clusters")

---------

# SKLearn K Means Implementation

Let's look into some of the additional features and benefits that sklearn's implementation gives us.

* Inertia
* Multiple runs of kmeans
* KMeans++
* Stopping conditions
* Transformations of data into cluster space [ BE CAREFUL HERE, gives euclidean, not Squared euclidean distance ]

In [ ]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=3,n_init=1,random_state=0)

km.fit(X)

km_clusters = km.predict(X)

draw2DClusters(X,km_clusters,"SKLearn Clusters")

In [ ]:
print km.inertia_
print np.sum(np.power([km.transform(X)[idx,x] for idx,x in enumerate(km_clusters)],2))

# Plotting out Inertia, Avg Dist, Cohesion, Separation

In [ ]:
inertia = []
avgDist = []
cohesion = []
separation = []

for n_k in range(1,20):
    km = KMeans(n_clusters=n_k)
    km.fit(X)
    
    inertia.append(km.inertia_)
    
    distToCenters = np.power([km.transform(X)[idx,x] for idx,x in enumerate(km.labels_)],2)
    avgDist.append( np.mean(distToCenters) )

    clust_coh = []
    for c in np.unique(km.labels_):
        clust_coh.append( np.sum(np.power([km.transform(X)[idx,x] for idx, x in enumerate(km.labels_) if x == c],2) ))

    cohesion.append( np.mean( clust_coh) )
    
    clust_sep = []
    for i in range(km.n_clusters):
        for j in range(i+1,km.n_clusters):
            clust_sep.append(np.power(distance.euclidean(km.cluster_centers_[i],km.cluster_centers_[j]),2))
    separation.append( np.mean(clust_sep))

    

n_k_range = range(1,20)
plt.figure(figsize=(10,10))

plt.subplot(2,2,1)
plt.plot(n_k_range,inertia)
plt.title("Total Inertia")

plt.subplot(2,2,2)
plt.plot(n_k_range,avgDist)
plt.title("Average dist To Cluster")

plt.subplot(2,2,3)
plt.plot(n_k_range,cohesion)
plt.title("Average Cluster Cohesion")

plt.subplot(2,2,4)
plt.plot(n_k_range,separation)
plt.title("Average Cluster Separation")